# Problem Statement
To predict the presence of perceptual brand attributes in the images that consumers post online. 

### Please refer to notebook Train_Validation_Test_Split for more details about problem statement.

This notebook expects that user has already run Train_Validation_Test_Split to preprocess the training data. 

Since we are using one vs rest approach to solve the multiclass problem, we need to create 4 classfiers, one for each attribute. Each classifier needs to have its own dataset, with one attribute beling classfied as positive, and all other images being classified as negative. 

Since training dataset is quite large, and we cannot load 4 datasets into memory at the same time, we need to run this notebook 4 times to create/train 4 classifiers. 

## Importing Libraries

In [ ]:
import tensorflow as tf
physical_devices = tf.config.list_physical_devices('GPU') 
try: 
  print('set memory growth')
  tf.config.experimental.set_memory_growth(physical_devices[0], True) 
except: 
  # Invalid device or cannot modify virtual devices once initialized. 
  pass 

In [ ]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle


import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Dense
import keras
from keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, InputLayer
from keras.models import Sequential
from keras import optimizers


from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

import matplotlib.pyplot as plt
import cv2
import os


In [ ]:
IMG_WIDTH=300
IMG_HEIGHT=300
IMG_DIM = (IMG_WIDTH, IMG_HEIGHT)

## Load training and validation data. 
Change the training/validation data file name corresponding to the model being trained. We need tp train 4 models, and for each model we need to use different dataset.  

In [ ]:
X_train = np.load('5000//X_train_0.npy')
X_val = np.load('5000//X_val_0.npy')

y_train = np.load('5000//y_train_0.npy')
y_val = np.load('5000//y_val_0.npy')

In [ ]:
X_val[0:2]

## Load the image into ram

In [ ]:
X_train= X_train.flatten()
X_train = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in X_train.tolist()]
X_train = np.array(X_train)
y_train = np.array(y_train)
X_val=   X_val.flatten()
X_val = [img_to_array(load_img(img, target_size=IMG_DIM)) for img in X_val.tolist()]
X_val = np.array(X_val)

y_val = np.array(y_val)
print( X_train.shape)
print( y_train.shape)
print( X_val.shape)
print( y_val.shape)

## Model Implementation
 - We are going to use pre trained resnet model 
 - do not want to load the last fully connected layers which act as the classifier 
 - Freeze the weights of the model by setting trainable as false
 - add our own fully connected layers on top of the ResNet50 model for our task-specific classification.



In [ ]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(IMG_HEIGHT,IMG_WIDTH,3))
# We do not want to load the last fully connected layers which act as the classifier we can add our 
# own fully connected layers on top of the ResNet50 model for our task-specific classification.

output = restnet.layers[-1].output
output = keras.layers.Flatten()(output)
restnet = Model(restnet.input, output=output)

#freeze the weights of the model by setting trainable as “False”
restnet.trainable = True
set_trainable = False
    
for layer in restnet.layers:
    if layer.name in ['res5c_branch2b', 'res5c_branch2c', 'activation_97']:
        set_trainable = True
    if set_trainable:
        layer.trainable = False
    else:
        layer.trainable = False
restnet.summary()

## Add task specific fully connected layer to fine tune the model for classification purpose

In [ ]:

input_shape=(IMG_HEIGHT,IMG_WIDTH,3)

model = Sequential()
model.add(restnet)
model.add(Dense(64, activation='relu', input_dim=input_shape))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['accuracy'])
model.summary()


## Training

In [ ]:
history =      model.fit(X_train,
                        y_train,
                        epochs=1,
                        batch_size = 30,
                        validation_data=(X_val,y_val) ,
                        verbose=1)

## Visualisation

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss']) 
plt.plot(history.history['val_loss']) 
plt.title('Model loss') 
plt.ylabel('Loss') 
plt.xlabel('Epoch') 
plt.legend(['Train', 'Test'], loc='upper left') 
plt.show()

## Save Model
We need to save 4 models. As we cannot train the model simultaneously because of gpu memory limitations, we need to run this notebook 4 times, to train each model independently. Please change the model name corresponding to the attribute being trained. 

In [ ]:
model.save('./savemodels/healthy_model.h5')

In [ ]:
model.save_weights('./checkpoints/my_checkpoint')